In [2]:
#Import the required libraries
import os
import cv2
import numpy as np
import pandas as pd
import pytesseract

#Natural Language Processing libraries
from spacy.matcher import PhraseMatcher
import nltk
import re as re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

In [3]:
# Importing tesseract from the location
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe" 

In [4]:
#Testing on one image
img = cv2.imread('./Dataset/Test957.jpg',cv2.IMREAD_GRAYSCALE)
height, width = img.shape[0], img.shape[1]

scale = 2000/width

img = cv2.resize(img, (0,0), fx=scale, fy=scale)
img_rm = cv2.medianBlur(img,5)
img_enc = cv2.adaptiveThreshold(img_rm,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,21)
text = pytesseract.image_to_string(img, lang='eng')

print(text)

UUM

ANDERS ON COOPER


In [18]:
import cv2
import os
import glob
img_dir = 'C:/Users/kanum/Desktop/Data_Science_Everyday/Hackathons/Love is Love/Dataset' # Enter Directory of all images 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
data = []
filename = []
for f1 in files:
    file = os.path.basename(f1)
    filename.append(file)
    img = cv2.imread(f1)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) #converting to gray image
    gray_rm = cv2.medianBlur(gray,5) #removing noise
    img_cont = cv2.adaptiveThreshold(gray_rm,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,21) #ENHANCING contrast
    text = pytesseract.image_to_string(img_cont, lang='eng')
    data.append(text)

In [19]:
inp = pd.DataFrame()
inp['Filename'] = filename
inp['text'] = data
inp.head(5)

,Filename,text
0,Test100.jpg,
1,Test1001.jpg,"Whea peogle ask\nwhat I ace in you,\n\nI just ..."
2,Test1012.jpg,
3,Test1022.jpg,
4,Test103.jpg,‘fy\nAoi\n1O/\n\nI MRESCHATLATESICLIS


I have extracted text from the images, we have performed OCR on images(I didn't preprocess the images yet). Now I can start performing Natural Language Processing on the extracted text.For extracting sentiment from the text I am implementing VADAR (Valence Aware Dictionary and sEntiment Reasoner)

__VADER provides a positive, negative, and neutral sentiment scalar, as well as a compound sentiment vector that combines the three__

In [26]:
analyzer = SentimentIntensityAnalyzer()

def clean_text(text):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', text).split())

inp["text"] = inp['text'].apply(lambda x: clean_text(x))
inp

def analyze_sentiment(text):
    analysis = TextBlob(text)
    
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity ==0:
        return 'Neutral'
    else:
        return 'Negative'

inp["Category"] = inp["text"].apply(lambda x: analyze_sentiment(x))

print(inp.head(5))

       Filename                                               text  Category
0   Test100.jpg                                                      Neutral
1  Test1001.jpg  Whea peogle ask what I ace in you I just smite...  Positive
2  Test1012.jpg                                                      Neutral
3  Test1022.jpg                                                      Neutral
4   Test103.jpg                     fy Aoi 1O I MRESCHATLATESICLIS   Neutral


In [22]:
submission = inp.drop(['text'],axis = 1)
submission = submission.replace(['old value'],'new value')
submission['Category'] = submission['Category'].replace(['Neutral'],'Random')
submission

,Filename,Category
0,Test100.jpg,Random
1,Test1001.jpg,Positive
2,Test1012.jpg,Random
3,Test1022.jpg,Random
4,Test103.jpg,Random
...,...,...
234,Test945.jpg,Random
235,Test946.jpg,Positive
236,Test957.jpg,Negative
237,Test979.jpg,Random


In [23]:
submission.to_csv("img_contrast.csv",index = False)